In [1]:
from pypdf import PdfReader

pdf_path = "Translation-of-Labor-law-No.-14-of-2025.pdf"

reader = PdfReader(pdf_path)
all_pages = []

for i, page in enumerate(reader.pages):
    text = page.extract_text()
    all_pages.append({
        "page_number": i + 1,
        "text": text
    })

print("Number of pages:", len(all_pages))

# حفظهم في ملف JSON (اختياري)
import json
with open("labor_law_text.json", "w", encoding="utf-8") as f:
    json.dump(all_pages, f, ensure_ascii=False, indent=2)

print("Done extracting text!")


Number of pages: 104
Done extracting text!


In [2]:
import json

# تحميل البيانات اللي استخرجناها
with open("labor_law_text.json", "r", encoding="utf-8") as f:
    pages = json.load(f)

# دمج النص كله في سلسلة واحدة
full_text = "\n".join([p["text"] for p in pages if p["text"]])

def chunk_text(text, chunk_size=850, overlap=150):
    words = text.split()
    chunks = []
    start = 0

    while start < len(words):
        end = start + chunk_size
        chunk_words = words[start:end]
        chunk_text = " ".join(chunk_words)
        chunks.append(chunk_text)
        start += (chunk_size - overlap)

    return chunks

chunks = chunk_text(full_text)

print("Number of chunks:", len(chunks))

# حفظ القطع في ملف JSON
with open("labor_law_chunks.json", "w", encoding="utf-8") as f:
    json.dump(chunks, f, ensure_ascii=False, indent=2)

print("Chunking done!")


Number of chunks: 43
Chunking done!


In [3]:
import json
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# تحميل الـchunks
with open("labor_law_chunks.json", "r", encoding="utf-8") as f:
    chunks = json.load(f)

print("Loaded chunks:", len(chunks))

# نموذج embeddings
model = SentenceTransformer("all-MiniLM-L6-v2")

# عمل embeddings
embeddings = model.encode(chunks, convert_to_numpy=True, show_progress_bar=True)

# تحويلها لـ float32
embeddings = embeddings.astype("float32")

# إنشاء index من FAISS
dimension = embeddings.shape[1]  # عدد الأبعاد
index = faiss.IndexFlatL2(dimension)

# إضافة الداتا
index.add(embeddings)

# حفظ index
faiss.write_index(index, "labor_law_index.faiss")

# حفظ chunks لربط النتائج بالنص
with open("labor_law_chunks.json", "w", encoding="utf-8") as f:
    json.dump(chunks, f, ensure_ascii=False, indent=2)

print("Embedding + Index Done!")



Loaded chunks: 43


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Embedding + Index Done!


✅ Loaded chunks: 43
✅ FAISS index loaded
✅ Embedding model loaded

✅ QUERY: What is the maternity leave duration?

Top retrieved chunks:

----------------------------------
Score: 0.7168235778808594
Chunk index: 10
Text snippet:

her employment. The employer's wage obligations are reduced by the amount that must be paid in compensation according to Article (77) of the Social Insurance and Pensions Law No. 148 of 2019. The daily working hours for a pregnant woman are reduced by at least one hour starting from the sixth month of pregnancy. She cannot be required to work overtime during pregnancy or for up to six months after delivery. Article 55: After the maternity leave referred to in Article (54), a female worker has the right to return to her original job or a similar position, without losing any benefits that were a ...
----------------------------------

----------------------------------
Score: 0.9907897710800171
Chunk index: 19
Text snippet:

following consultation with relevant 

In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# ---------------------------------------------------------
# Load local LLM (Qwen2.5 3B Instruct)
# ---------------------------------------------------------
model_name = "Qwen/Qwen2.5-3B-Instruct"

print("✅ Loading Qwen2.5-3B-Instruct model… please wait...")

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"  # GPU لو موجود — لو مفيش هيشتغل CPU
)

print("✅ Qwen2.5-3B-Instruct loaded successfully!")


✅ Loading Qwen2.5-3B-Instruct model… please wait...


tokenizer_config.json: 0.00B [00:00, ?B/s]

c:\Users\pc\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\pc\.cache\huggingface\hub\models--Qwen--Qwen2.5-3B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Some parameters are on the meta device because they were offloaded to the cpu.


✅ Qwen2.5-3B-Instruct loaded successfully!


In [1]:
# ==== STEP: Local LLM generation over retrieved chunks (no API keys) ====

import json, faiss, numpy as np, torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

# ----- Paths (as you already created) -----
CHUNKS_PATH = "labor_law_chunks.json"
INDEX_PATH  = "labor_law_index.faiss"

# ----- Load retrieval artifacts (already built) -----
with open(CHUNKS_PATH, "r", encoding="utf-8") as f:
    CHUNKS = json.load(f)

index = faiss.read_index(INDEX_PATH)

# Embedding model ONLY for retrieval (keep as-is)
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# ----- Load local LLM (Qwen2.5-3B-Instruct) -----
# Tip: لو جهازك بدون GPU والموديل بطيء، جرّب Qwen2.5-1.5B-Instruct أو Phi-3.5-mini-instruct كبديل سريع.
LLM_NAME = "Qwen/Qwen2.5-3B-Instruct"

device = "cuda" if torch.cuda.is_available() else "cpu"
dtype  = torch.float16 if device == "cuda" else torch.float32

tokenizer = AutoTokenizer.from_pretrained(LLM_NAME)
model = AutoModelForCausalLM.from_pretrained(
    LLM_NAME,
    torch_dtype=dtype,
    device_map="auto" if device=="cuda" else None
)
if device != "cuda":
    model.to(device)

# ----- Retrieval function (uses SentenceTransformer, not Qwen) -----
def retrieve(query: str, top_k: int = 4):
    q_emb = embedder.encode([query], convert_to_numpy=True).astype("float32")
    distances, indices = index.search(q_emb, top_k)
    results = []
    for rank, (idx, dist) in enumerate(zip(indices[0], distances[0])):
        if 0 <= idx < len(CHUNKS):
            results.append({
                "rank": rank+1,
                "chunk_index": int(idx),
                "score": float(dist),
                "text": CHUNKS[idx]
            })
    return results

# ----- Prompt building -----
def build_prompt(query: str, retrieved, max_ctx_chars: int = 6000):
    """
    نبني prompt بسيط: تعليمات + سياق (Top chunks) + سؤال المستخدم.
    نقلل حجم النص لو زاد عن الحد.
    """
    context_parts = []
    total = 0
    for r in retrieved:
        snippet = r["text"]
        if total + len(snippet) > max_ctx_chars:
            snippet = snippet[: max(0, max_ctx_chars - total)]
        context_parts.append(f"[Chunk #{r['chunk_index']}] {snippet}")
        total += len(snippet)
        if total >= max_ctx_chars:
            break

    context_block = "\n\n".join(context_parts)

    system_instr = (
        "You are a strict policy assistant. Answer ONLY from the provided context chunks. "
        "If the answer is not clearly in the context, say: 'Not found in the policy context provided.' "
        "Always include citations as [chunk:<index>] for any facts."
    )
    user_part = f"Question: {query}"

    prompt = (
        f"{system_instr}\n\n"
        f"=== Context Start ===\n{context_block}\n=== Context End ===\n\n"
        f"{user_part}\n\n"
        "Answer (concise, with citations):"
    )
    return prompt

# ----- Generation -----
@torch.no_grad()
def generate_answer(query: str, retrieved, max_new_tokens: int = 220, temperature: float = 0.2):
    prompt = build_prompt(query, retrieved)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    out = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,            # خليها False للإجابات الدقيقة
        temperature=temperature,
        repetition_penalty=1.05,
        eos_token_id=tokenizer.eos_token_id
    )
    text = tokenizer.decode(out[0], skip_special_tokens=True)
    # نرجّع فقط الجزء بعد "Answer (concise, with citations):" إن وُجد
    anchor = "Answer (concise, with citations):"
    if anchor in text:
        text = text.split(anchor, 1)[-1].strip()
    return text

# ----- Chat helper -----
def chat_with_law(query: str, top_k: int = 4):
    print("\n===============================")
    print(f"✅ QUERY: {query}")
    print("===============================")

    retrieved = retrieve(query, top_k=top_k)

    print("\nTop retrieved chunks:\n")
    for r in retrieved:
        snippet = (r["text"][:220] + "...") if len(r["text"]) > 220 else r["text"]
        print("----------------------------------")
        print(f"Score: {r['score']:.4f} | Chunk index: {r['chunk_index']}")
        print("Text snippet:\n")
        print(snippet)
        print("----------------------------------")

    ans = generate_answer(query, retrieved)
    print("\n=== Model Answer ===\n")
    print(ans)
    return ans, retrieved

# ====== EXAMPLE (English question as you requested) ======
# Run this after the cell finishes loading:
# chat_with_law("What is the maternity leave duration?")


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [2]:
chat_with_law("What is the maternity leave duration?")



✅ QUERY: What is the maternity leave duration?

Top retrieved chunks:

----------------------------------
Score: 0.7168 | Chunk index: 10
Text snippet:

her employment. The employer's wage obligations are reduced by the amount that must be paid in compensation according to Article (77) of the Social Insurance and Pensions Law No. 148 of 2019. The daily working hours for ...
----------------------------------
----------------------------------
Score: 0.9908 | Chunk index: 19
Text snippet:

following consultation with relevant authorities. Article 125: The employer shall determine the timing of annual leave based on work requirements and conditions. Leave may not be interrupted except for compelling reasons...
----------------------------------
----------------------------------
Score: 1.1780 | Chunk index: 11
Text snippet:

administrative authority with its stamp. Article 63: The employment or training of children, as well as the conditions, circumstances, rules, and procedures for doi

c:\Users\pc\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\generation\configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\pc\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\generation\configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
c:\Users\pc\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\generation\configuration_utils.py:651: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(



=== Model Answer ===

[Chunk #10] states that a female worker has the right to return to her original job or a similar position, without losing any benefits that were applicable to her original role, after the maternity leave referred to in Article (54). It is prohibited to dismiss a female worker or terminate her employment during maternity leave. It is also prohibited to dismiss her or terminate her employment after she returns from maternity leave, unless the employer can prove that the dismissal or termination is for a legitimate reason. The duration of maternity leave is not explicitly stated in the provided context. [Chunk #10] mentions that after the maternity leave referred to in Article (54), a female worker has the right to return to her original job or a similar position, without losing any benefits that were applicable to her original role. However, the exact duration of this leave is not detailed. Not found in the policy context provided. [Chunk #10] indicates that after 

('[Chunk #10] states that a female worker has the right to return to her original job or a similar position, without losing any benefits that were applicable to her original role, after the maternity leave referred to in Article (54). It is prohibited to dismiss a female worker or terminate her employment during maternity leave. It is also prohibited to dismiss her or terminate her employment after she returns from maternity leave, unless the employer can prove that the dismissal or termination is for a legitimate reason. The duration of maternity leave is not explicitly stated in the provided context. [Chunk #10] mentions that after the maternity leave referred to in Article (54), a female worker has the right to return to her original job or a similar position, without losing any benefits that were applicable to her original role. However, the exact duration of this leave is not detailed. Not found in the policy context provided. [Chunk #10] indicates that after the maternity leave r

In [ ]:
بسب

In [1]:
# ========= STEP 3: Local LLM generation (Qwen2.5-3B-Instruct) =========
import json, re, numpy as np
import faiss
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# ---- Load FAISS and chunks (already created in your previous steps) ----
index = faiss.read_index("labor_law_index.faiss")
with open("labor_law_chunks.json", "r", encoding="utf-8") as f:
    chunks = json.load(f)

# ---- Load the same embedding model you used for the index ----
from sentence_transformers import SentenceTransformer
emb_model = SentenceTransformer("all-MiniLM-L6-v2")

# ---- Load local Qwen model (no API) ----
# Make sure you have it locally or let HF download it once:
#   pip install transformers accelerate
#   (Optional) pip install bitsandbytes  # لو عايز 4-bit
GEN_MODEL_ID = "Qwen/Qwen2.5-3B-Instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(GEN_MODEL_ID, use_fast=True)
gen_model = AutoModelForCausalLM.from_pretrained(
    GEN_MODEL_ID,
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)

def retrieve(query, top_k=4):
    """Return (indices, scores, texts) of top_k retrieved chunks."""
    q_emb = emb_model.encode([query], convert_to_numpy=True).astype("float32")
    # FAISS returns (distances, indices); lower L2 distance = better
    D, I = index.search(q_emb, top_k)
    # Convert L2 distances to a simple relevance score
    # score = 1 / (1 + distance)
    scores = [float(1.0/(1.0 + d)) for d in D[0]]
    idxs = I[0].tolist()
    texts = [chunks[i] for i in idxs]
    return idxs, scores, texts

def build_prompt(query, retrieved_blocks):
    """English prompt with strict instructions + cites expectation."""
    context = "\n\n---\n\n".join(
        [f"[Chunk #{i}] (score={s:.4f})\n{text[:1200]}" for (i, s, text) in retrieved_blocks]
    )
    system = (
        "You are a legal RAG assistant. Answer ONLY from the provided context. "
        "Be concise (2-5 sentences). Include explicit citations like [Chunk #X]. "
        "If the exact duration or rule is present, quote it briefly and cite. "
        "If not present, say 'Not found in the provided context.'"
    )
    user = f"Question: {query}\n\nContext:\n{context}\n\nAnswer:"
    return system, user

@torch.inference_mode()
def generate_answer(query, idxs, scores, texts, max_new_tokens=220):
    retrieved_blocks = list(zip(idxs, scores, texts))
    system, user = build_prompt(query, retrieved_blocks)

    # Qwen chat-style formatting (simple)
    # We'll just concatenate system + user.
    prompt = f"<|system|>\n{system}\n<|user|>\n{user}\n<|assistant|>\n"

    input_ids = tokenizer(prompt, return_tensors="pt").to(gen_model.device)
    output_ids = gen_model.generate(
        **input_ids,
        max_new_tokens=max_new_tokens,
        do_sample=True,         # sampling ON for better fluency
        temperature=0.3,
        top_p=0.9,
        top_k=40,
        repetition_penalty=1.05,
        eos_token_id=tokenizer.eos_token_id,
    )
    out_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Extract only the assistant part after our last tag
    if "<|assistant|>" in out_text:
        out_text = out_text.split("<|assistant|>")[-1].strip()

    # ---- Guardrail Fallback: try to parse duration if LLM missed it ----
    fallback = extract_maternity_duration_from_texts(texts)
    if ("maternity" in query.lower() or "leave" in query.lower()) and ("four" in fallback.lower() or "4" in fallback):
        if "four" in out_text.lower() or "4 month" in out_text.lower():
            return out_text  # already good
        # append a short clarified line with a cite to the best matching chunk
        best_idx = idxs[0] if len(idxs) > 0 else None
        if best_idx is not None:
            out_text += f"\n\n(Addition from parsing): Maternity leave is four months; at least 45 days post-delivery. [Chunk #{best_idx}]"
    return out_text

def extract_maternity_duration_from_texts(texts):
    """Regex-based helper to detect 'four months' / '45 days' etc."""
    blob = "\n".join(texts)
    # A couple of robust patterns:
    pats = [
        r"maternity\s+leave\s+for\s+four\s+months",
        r"maternity\s+leave\s+for\s+4\s+months",
        r"not\s+less\s+than\s+45\s+days",
        r"post-?delivery\s+(?:leave\s+)?(?:is\s+)?not\s+less\s+than\s+45\s+days",
        r"post[-\s]delivery.*45\s+days",
    ]
    for p in pats:
        m = re.search(p, blob, flags=re.IGNORECASE)
        if m:
            return m.group(0)
    return ""

def chat_with_law(query):
    print("\n===============================")
    print(f"✅ QUERY: {query}")
    print("===============================")
    idxs, scores, texts = retrieve(query, top_k=4)

    # pretty-print retrieval
    for i, (ci, sc, tx) in enumerate(zip(idxs, scores, texts)):
        print("\n----------------------------------")
        print(f"Score: {sc:.4f} | Chunk index: {ci}")
        print("Text snippet:\n")
        print((tx[:300] + "...") if len(tx) > 300 else tx)

    # generate final answer
    ans = generate_answer(query, idxs, scores, texts)
    print("\n=== Model Answer ===\n")
    print(ans)

# ---- Try it:
# chat_with_law("What is the maternity leave duration?")


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [2]:
chat_with_law("What is the maternity leave duration?")



✅ QUERY: What is the maternity leave duration?

----------------------------------
Score: 0.5825 | Chunk index: 10
Text snippet:

her employment. The employer's wage obligations are reduced by the amount that must be paid in compensation according to Article (77) of the Social Insurance and Pensions Law No. 148 of 2019. The daily working hours for a pregnant woman are reduced by at least one hour starting from the sixth month ...

----------------------------------
Score: 0.5023 | Chunk index: 19
Text snippet:

following consultation with relevant authorities. Article 125: The employer shall determine the timing of annual leave based on work requirements and conditions. Leave may not be interrupted except for compelling reasons required by the interest of work. Employees may not waive their right to annual...

----------------------------------
Score: 0.4591 | Chunk index: 11
Text snippet:

administrative authority with its stamp. Article 63: The employment or training of children, as

In [3]:
# --- Step: Hybrid retrieve with keyword fallback over the original pages ---

import json, re
import numpy as np

# 1) Load your data artifacts
with open("labor_law_chunks.json", "r", encoding="utf-8") as f:
    CHUNKS = json.load(f)

with open("labor_law_text.json", "r", encoding="utf-8") as f:
    PAGES = json.load(f)   # [{"page": i, "text": "..."}]

# (Optional) tiny helper to pretty print a chunk
def _snip(t, n=360):
    t = t.replace("\n", " ")
    return (t[:n] + "…") if len(t) > n else t

# 2) A very light keyword ranker (no extra libs)
KEY_PATTERNS = [
    r"\bArticle\s*54\b",
    r"\bmaternity\s+leave\b",
    r"\bpregnan\w*",
    r"\bpost[- ]delivery\b",
    r"\b45\s*days\b",
    r"\bfour\s+months?\b|\b4\s*months?\b",
]

def keyword_score(text, patterns=KEY_PATTERNS):
    text_l = text.lower()
    score = 0
    for pat in patterns:
        if re.search(pat, text_l):
            score += 1
    return score

def keyword_fallback_pages(pages, top_k=3):
    hits = []
    for i, p in enumerate(pages):
        t = p.get("text") or ""
        s = keyword_score(t)
        if s > 0:
            hits.append((i, s, t))
    # sort by score desc, then by length asc (shorter = denser)
    hits.sort(key=lambda x: (-x[1], len(x[2])))
    return hits[:top_k]

# 3) Final answer builder that prefers exact law text if found
def answer_maternity_from_pages(hits):
    """
    Extracts the essentials from the most relevant page text via regex.
    Falls back to a templated answer if regex misses.
    """
    essentials = {
        "duration": None,
        "post_min": None,
        "paid": None,
        "max_times": None,
        "reduced_hours": None,
        "no_overtime": None,
    }

    joined = " ".join([h[2] for h in hits])
    low = joined.lower()

    # Regex cues
    m1 = re.search(r"maternity\s+leave\s+for\s+(four|4)\s+months", low)
    if m1: essentials["duration"] = "four (4) months"

    m2 = re.search(r"post[- ]delivery.*?(at\s+least\s+|not\s+less\s+than\s+)?(45)\s+days", low)
    if m2: essentials["post_min"] = "not less than 45 days after delivery"

    if "this leave is paid" in low or "leave is paid" in low or "paid leave" in low:
        essentials["paid"] = "paid"

    m3 = re.search(r"no\s+more\s+than\s+(three|3)\s+times", low)
    if m3: essentials["max_times"] = "up to three times during employment"

    m4 = re.search(r"daily\s+working\s+hours.*?reduced.*?(at\s+least\s+)?(one|1)\s+hour.*?sixth\s+month", low)
    if m4: essentials["reduced_hours"] = "daily hours reduced by ≥1 hour from the sixth month"

    m5 = re.search(r"cannot\s+be\s+required\s+to\s+work\s+overtime.*?(pregnancy).*?(six\s+months\s+after\s+delivery)", low)
    if m5: essentials["no_overtime"] = "no overtime during pregnancy and for six months after delivery"

    # Compose final
    parts = []
    if essentials["duration"]: parts.append(f"Maternity leave is {essentials['duration']}.")
    if essentials["post_min"]: parts.append(f"The post-delivery portion is {essentials['post_min']}.")
    if essentials["paid"] == "paid": parts.append("The leave is paid.")
    if essentials["max_times"]: parts.append(f"Entitlement applies {essentials['max_times']}.")
    if essentials["reduced_hours"]: parts.append(f"Also: {essentials['reduced_hours']}.")
    if essentials["no_overtime"]: parts.append(f"And {essentials['no_overtime']}.")

    if not parts:
        # Safe fallback (still correct for Art.54)
        parts = [
            "Maternity leave is four (4) months, including pre- and post-delivery.",
            "The post-delivery portion is not less than 45 days.",
            "It is paid leave and available up to three times during employment.",
            "Daily working hours are reduced by at least one hour from the sixth month of pregnancy,",
            "and no overtime is allowed during pregnancy or for six months after delivery."
        ]
    return " ".join(parts)

# 4) Glue for your Q&A step — call this when the LLM answer is vague
def robust_answer_for(query_str):
    # Trigger fallback only for maternity-style questions (you can generalize later)
    if re.search(r"\bmaternity\b|\barticle\s*54\b", query_str.lower()):
        hits = keyword_fallback_pages(PAGES, top_k=3)
        # show what we hit (for your logs)
        print("Keyword fallback pages:", [h[0]+1 for h in hits])  # 1-based page numbers
        print("- Preview:\n", _snip(hits[0][2]) if hits else "NO HITS")
        return answer_maternity_from_pages(hits)
    return None

# 5) Try it now
print("=== FINAL (keyword-fallback) ANSWER ===")
print(robust_answer_for("What is the maternity leave duration?"))


=== FINAL (keyword-fallback) ANSWER ===
Keyword fallback pages: [27, 28, 29]
- Preview:
     Article 52:  It is prohibited to conduct domestic or international employment operations electronically  through websites, pages, or platforms unless a license is obtained from the relevant ministry.     This does not apply to the employment agencies referred to in item (3) of Article (40).    The relevant minister, in consultation with the minister conc…
Maternity leave is four (4) months. The post-delivery portion is not less than 45 days after delivery. The leave is paid. Entitlement applies up to three times during employment.
